In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
pd.options.display.max_columns, pd.options.display.max_rows = 500, 200
pd.options.display.float_format = '{:.1f}'.format
import warnings
warnings.filterwarnings('ignore')
import joblib

from euro_soccer.preprocessing import DataPreprocessor
from euro_soccer.glicko_soccer import GlickoSoccer

In [2]:
matches = pd.read_csv('matches.csv')
matches = DataPreprocessor(is_actual_draw_predictions=True).preprocessing(matches)
matches.tail(1)

[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] lambda_l2 is set=5.648, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.648
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] lambda_l1 is set=0.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0


,index,date,home_team,away_team,country,tournament,season,tournament_type,outcome,is_pandemic,draw_probability
130651,309702,2022-12-31,H. Raanana,Ramat Hasharon,Israel,Israel. Leumit League,2021,2,H,0,0.3


In [12]:
count_matches, count_seasons, count_tournaments = DataPreprocessor().test_data(matches)

In [13]:
count_matches

,country,tournament_type,season,number_matches


In [14]:
count_seasons

,country,tournament_type,number_seasons
59,Europa Conference League,3,1
60,Europa Conference League,4,1


In [15]:
count_tournaments

,country,number_tournaments,tournament_type


In [16]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115670 entries, 0 to 130651
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   index             115670 non-null  int64         
 1   date              115670 non-null  datetime64[ns]
 2   home_team         115670 non-null  object        
 3   away_team         115670 non-null  object        
 4   country           115670 non-null  object        
 5   tournament        115670 non-null  object        
 6   season            115670 non-null  int64         
 7   tournament_type   115670 non-null  int64         
 8   outcome           115670 non-null  object        
 9   is_pandemic       115670 non-null  int64         
 10  draw_probability  115670 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(5)
memory usage: 10.6+ MB


In [22]:
# matches['draw_probability'].mean()

In [23]:
# # matches.loc[(matches['home_team'].str.contains('B')) & (matches['away_team'].str.contains('AS Roma'))]

In [24]:
# matches.info()

In [25]:
# matches['season'].value_counts()

In [26]:
# matches.loc[matches['tournament'].str.contains('Con')]

In [27]:
# matches.loc[matches['country'].str.contains('Uz')]

# Fit Params

In [ ]:
glicko = GlickoSoccer() # 38773 38756
league_params = glicko.fit_params(matches, 100, is_params_initialization=False)

Current Loss: 27552.243972951943
############################################################################

Best Draw Parameter: -0.5537

############################################################################

Best Volatility Parameter: 0.10400000000000001



# Ratings

In [24]:
glicko = GlickoSoccer()
league_params = joblib.load('data/league_params.pkl')
# league_params = glicko._league_params_initialization(matches)
ratings = glicko.rate_teams(matches, league_params)
ratings = glicko.ratings_to_df(ratings, matches)
leagues_ratings = glicko.league_ratings(ratings, number_top_teams=5)
# first_leagues = set(matches.loc[matches['tournament_type'] == 1, 'tournament'])
# leagues_ratings = leagues_ratings.loc[leagues_ratings['league'].isin(first_leagues)]
leagues_ratings

,#,league,rating
0,1,England. Premier League,2093.0
1,2,Italy. Serie A,2021.2
2,3,Spain. LaLiga,2021.1
3,4,Portugal. Primeira Liga,1984.5
4,5,Germany. Bundesliga,1981.8
5,6,France. Ligue 1,1958.9
6,7,Netherlands. Eredivisie,1948.3
7,8,Faroe Islands. Premier League,1812.7
8,9,Scotland. Premiership,1808.5
9,10,Belgium. Jupiler Pro League,1804.3


In [8]:
ratings.head(50)

,#,team,rating,league
0,1,Liverpool,2126.8,England. Premier League
1,2,Bayern Munich,2100.9,Germany. Bundesliga
2,3,Manchester City,2099.0,England. Premier League
3,4,FC Porto,2069.3,Portugal. Primeira Liga
4,5,Real Madrid,2064.1,Spain. LaLiga
5,6,Ajax,2038.0,Netherlands. Eredivisie
6,7,Inter,2024.6,Italy. Serie A
7,8,Chelsea,2008.4,England. Premier League
8,9,Sporting,2008.0,Portugal. Primeira Liga
9,10,Crvena zvezda,2007.7,Serbia. Super Liga


# Skellam

In [9]:
# from euro_soccer.draw_model import DrawLightGBM

DrawLightGBM().cross_val_score()

OSError: dlopen(/Users/andreyshelopugin/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/andreyshelopugin/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [5]:
# import joblib
train = joblib.load('data/train.pkl')
train.head()

,team,score,index,is_home,avg_scoring_10,avg_scoring_20,is_pandemic,mean_score_10,median_score_10,mean_score_20,mean_score_10_against,median_score_10_against,mean_score_20_against
129670,Den Bosch,2,69077,0,2.500000,2.5,1,1.800000,1.5,1.5,1.700000,1.0,2.0
4917,Dubnica,5,67608,1,1.444444,NaN,0,0.777778,1.0,1.0,3.222222,3.0,3.0
17732,Rapid Vienna,2,27656,1,2.400000,2.4,0,1.100000,0.5,1.0,0.600000,0.0,1.0
132205,Chambly,2,51277,0,1.200000,1.2,1,1.100000,0.5,1.0,0.800000,0.5,1.0
115031,Tobol,0,10064,0,1.300000,1.3,0,1.200000,1.0,1.0,0.300000,0.0,0.0


In [6]:
# train.loc[train['team'] == 'Manchester City']

,team,score,index,is_home,avg_scoring_10,avg_scoring_20,is_pandemic,mean_score_10,median_score_10,mean_score_20,mean_score_10_against,median_score_10_against,mean_score_20_against
99593,Manchester City,2,8107,0,2.8,2.8,0,1.700000,1.5,2.0,1.000000,1.0,0.5
97208,Manchester City,1,75728,0,2.7,2.7,0,1.900000,2.0,2.0,1.300000,1.5,1.0
101412,Manchester City,3,8079,0,2.6,2.6,0,1.700000,1.5,2.0,0.800000,0.5,0.5
22198,Manchester City,1,8075,1,1.5,1.5,0,3.300000,3.0,3.0,0.900000,0.5,1.0
112351,Manchester City,3,89736,0,1.7,1.7,0,2.500000,2.5,2.0,0.700000,0.5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139024,Manchester City,4,82632,0,1.4,1.4,1,1.100000,1.0,1.0,0.400000,0.0,0.5
51661,Manchester City,2,82643,1,3.1,3.1,1,2.900000,2.5,2.0,0.600000,0.0,1.0
89840,Manchester City,3,80587,0,2.6,2.6,0,1.666667,1.0,1.0,1.777778,1.0,2.0
16367,Manchester City,3,8163,1,2.2,2.2,0,3.200000,3.0,3.0,0.900000,1.0,1.0


In [7]:
# train['avg_scoring_10'].mean()

1.364716147298895